In [58]:
PROJECT = "train-to-cloud-city-4"
BASE = "http://localhost:8000/"

In [59]:
import requests
import json
from urllib.parse import urljoin


In [60]:
res = requests.get(urljoin(BASE,"service/"))
# print(res.text)

service_data = json.loads(res.text)
print(service_data["cloud-run"])


{'slug': 'cloud-run', 'name': 'Cloud Run', 'description': 'Serverless for containerized applications', 'doc_url': 'https://cloud.google.com/run/docs'}


In [61]:
from google.cloud import firestore
db = firestore.Client(project=PROJECT)

services_ref = db.collection("services")

q = services_ref.count()
#q??
print(q.get())

docs = services_ref.stream()

for i, doc in enumerate(docs):
    print(f"{doc.id} => {doc.to_dict()}")
    if i > 5: break

[[<Aggregation alias=field_1, value=248, readtime=2024-02-10 01:27:33.261544+00:00>]]
access-context-manager => {'slug': 'access-context-manager', 'description': 'Fine-grained, attribute based access-control', 'doc_url': 'https://cloud.google.com/access-context-manager/docs', 'name': 'Access Context Manager'}
access-transparency => {'slug': 'access-transparency', 'description': 'Audit cloud provider access', 'doc_url': 'https://cloud.google.com/cloud-provider-access-management/access-transparency/docs/overview', 'name': 'Access Transparency'}
admin-sdk => {'slug': 'admin-sdk', 'description': 'Manage Google Workspace resources', 'doc_url': 'https://developers.google.com/apps-script/advanced/admin-sdk-directory', 'name': 'Admin SDK'}
alloydb => {'slug': 'alloydb', 'description': 'Scalable & performant PostgreSQL - compatible DB', 'doc_url': 'https://cloud.google.com/alloydb/docs', 'name': 'AlloyDB'}
amp-for-email => {'slug': 'amp-for-email', 'description': 'Dynamic interactive email', 'd

In [62]:
# insert/update service data
for key, data in service_data.items():
    doc_ref = services_ref.document(key)
    doc_ref.set(data)

In [63]:
# add patterns

res = requests.get(urljoin(BASE,"pattern/"))
# print(res.text)
pattern_data = json.loads(res.text)

for pattern in pattern_data.values():
    # print(pattern)
    for checkpoint in pattern["checkpoints"]:
        for service in checkpoint["satisfying_services"]:
            if not ((service in service_data) or (service in {"*"})):
                print (f"'{service}' not found. Referenced in checkpoint: {checkpoint}")

patterns_ref = db.collection("patterns")
for data in pattern_data.values():
    doc_ref = patterns_ref.document(data["slug"])
    doc_ref.set(data)


In [64]:
res = requests.get(urljoin(BASE,"default_world/"))
# print(res.text)
default_world = json.loads(res.text)
default_world

global_ref = db.collection("global")
global_ref.document("world").set(default_world)


update_time {
  seconds: 1707528462
  nanos: 626107000
}

In [65]:
docs = services_ref.stream()
for i, doc in enumerate(docs):
    if i >= 5: break
    print(f"{doc.id} => {doc.to_dict()}")
print("----")
    
docs = patterns_ref.stream()
for i, doc in enumerate(docs):
    if i >= 5: break
    print(f"{doc.id} => {doc.to_dict()}")



access-context-manager => {'slug': 'access-context-manager', 'description': 'Fine-grained, attribute based access-control', 'doc_url': 'https://cloud.google.com/access-context-manager/docs', 'name': 'Access Context Manager'}
access-transparency => {'slug': 'access-transparency', 'description': 'Audit cloud provider access', 'doc_url': 'https://cloud.google.com/cloud-provider-access-management/access-transparency/docs/overview', 'name': 'Access Transparency'}
admin-sdk => {'slug': 'admin-sdk', 'description': 'Manage Google Workspace resources', 'doc_url': 'https://developers.google.com/apps-script/advanced/admin-sdk-directory', 'name': 'Admin SDK'}
alloydb => {'slug': 'alloydb', 'description': 'Scalable & performant PostgreSQL - compatible DB', 'doc_url': 'https://cloud.google.com/alloydb/docs', 'name': 'AlloyDB'}
amp-for-email => {'slug': 'amp-for-email', 'description': 'Dynamic interactive email', 'doc_url': 'https://developers.google.com/apps-script/add-ons/gmail', 'name': 'AMP for E